In [1]:
import cv2
import onnx

import sys
sys.path.append('..')
from models.scnn_pytorch.model import SCNN
from models.erfnet.erfnet import ERFNet
import torch
import os
import numpy as np

In [2]:
# load model
dummy_input = torch.rand(1, 3, 976, 208)
erf_net = ERFNet(5)
weights = torch.load('D:/Dev/Codes-for-Lane-Detection/ERFNet-CULane-PyTorch/trained/ERFNet_trained.tar')['state_dict']
weights = {k.replace("module.", ""):v for k,v in weights.items()}
erf_net.load_state_dict(weights)
erf_net.eval()

In [3]:
# export model
torch.onnx.export(erf_net, dummy_input, 'erf_net.onnx', export_params=True, opset_version=11, do_constant_folding=True,
                  input_names=['input'], output_names=['seg', 'prob'])

C:\Users\DHH\miniconda3\envs\ets2\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [3]:
# check exported model
onnx_model = onnx.load("erf_net.onnx")
onnx.checker.check_model(onnx_model, full_check=True)

In [4]:
net = cv2.dnn.readNetFromONNX('erf_net.onnx')

error: OpenCV(4.3.0) ..\modules\dnn\src\onnx\onnx_importer.cpp:262: error: (-204:Requested object was not found) Blob 566 not found in const blobs in function 'cv::dnn::dnn4_v20200310::ONNXImporter::getBlob'


In [ ]:
img = cv2.imread("D:/Dev/auto-ets2/images/1591496964382.png")

In [21]:
blob = cv2.dnn.blobFromImage(img, 1.0, (976, 208))
blob.shape

(1, 3, 208, 976)

In [24]:
net.setInput(blob)
output_names = net.getUnconnectedOutLayersNames()

In [27]:
outs = net.forward(output_names)

In [45]:
seg = outs[0]
seg.shape

(1, 5, 208, 976)

In [34]:
cv2.minMaxLoc(seg[0])

error: OpenCV(4.3.0) ..\modules\core\src\minmax.cpp:1565: error: (-215:Assertion failed) _img.dims() <= 2 in function 'cv::minMaxLoc'


In [7]:
cv2.__version__

'4.3.0'